### BWT Homework

Today we will construct BWT from scratch. First we will construct a suffix array for a given string using pysuffixarray.


In [1]:
!pip install pysuffixarray

from pysuffixarray.core import SuffixArray
sa = SuffixArray('ACAACG')
print(sa.suffix_array())

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pysuffixarray: filename=pysuffixarray-0.0.1-py3-none-any.whl size=3494 sha256=31ac767e79c584507aff4deb07ed38c72d5a7ca92644f429cd8a7d1cc30de9c7
  Stored in directory: c:\users\redmi\appdata\local\pip\cache\wheels\59\7b\6f\53c0a16911fde6bd99aee6beaebc988ad44dd6c385775f2183
Successfully built pysuffixarray
[6, 2, 0, 3, 1, 4, 5]



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Task 1: Create BWT using suffix array:

- Using BioPython upload SARS-CoV-2 reference genome from fasta file (genome.fa in BWT_folder)
- Construct suffix array
- Construct BWT from suffix array
- Don't forget to add special symbol (but after SA construction)

In [106]:
from Bio import SeqIO

rec = next(SeqIO.parse("BWT_folder/genome.fa", "fasta"))
sequence = str(rec.seq)

In [111]:
"""Есть собственное решение суффиксного массива за O(nlogn⋅time(less))"""


import math
string = sequence
# string = '99 bottles of beer.'

string += chr(1)


d = len(string)

n = math.ceil(math.log2(d)) + 1

p = [-1] * d
color = [-1] * d
def count_sort(ar):
    n = len(ar)
    pos = [0] * 97
    answer = [0] * n
    for v in ar:
        if v == chr(1):
            pos[0] += 1
            
        else:
            
            ind = ord(v) - ord(' ') + 1
            # print(ind)
            pos[ind] += 1
        # print(pos)

    for i in range(1, 97):
        pos[i] += pos[i-1]
    # print(pos)
    for i in range(n-1, -1, -1):
        if ar[i] == chr(1):
            answer[pos[0]-1] = i
            pos[0] -= 1
        else:
            # print(ord(ar[i]))
            ind = ord(ar[i]) - ord(' ') + 1
            answer[pos[ind]-1] = i
            pos[ind] -= 1
        # print(answer)
    return answer
pos = count_sort(string)
# print(pos)
color = [0] * d
c = 0
color[pos[0]] = 0
for i in range(1, d):
    if string[pos[i]] != string[pos[i-1]]:
        c += 1
    color[pos[i]] = c


for k in range(1, n+1):
    ar = []
    for i in range(d):
        j = pos[i]
        l = (j - (2**(k-1)) + d) % d
        ar.append([(color[l], color[j]), l])
    pos2 = [0] * d
    color2 = [0] * d
    p = [0] * d
    for i in range(d):
        c, ind = ar[i]
        c1, c2 = c
        p[c1] += 1
        # print(p)
    for i in range(1, d):
        p[i] += p[i-1]


    # print(ar)
    # print(p)

    for i in range(d-1, -1, -1):
        c, ind = ar[i]
        c1, c2 = c
        pos2[p[c1]-1] = ind
        p[c1] -= 1
        color2[ind] = c
    color2_cur = [0] * d
    c = 0
    
    for i in range(1, d):
        if color2[pos2[i]] != color2[pos2[i-1]]:
            c += 1
        color2_cur[pos2[i]] = c
    pos, color = pos2, color2_cur
# for i in pos:
#     s1, s2 = string[i:].split(chr(1))
#     if s1:
#         print(s1)
# answer = list(map(-1, pos[1:]))

bwt = []
for i in range(d):
    if pos[i] == 0:
        bwt.append(string[-1])
    else:
        bwt.append(string[pos[i] - 1])
print("Суффиксный массив со спецсимволом:")
suf = pos
print(*pos)
pos = list(map(lambda x: x+1, pos[1:]))
print("Суффиксный массив без спецсимвола:")
print(*pos)
print("BWT:")
print(''.join(bwt))

bwt1 = ''.join(bwt)



Суффиксный массив со спецсимволом:
29903 29902 29901 29900 29899 29898 29897 29896 29895 29894 29893 29892 29891 29890 29889 29888 29887 29886 29885 29884 29883 29882 29881 29880 29879 29878 29877 29876 29875 29874 29873 29872 29871 29870 1813 11990 23164 22000 13360 20135 23887 15983 16800 14000 25797 29051 13586 20445 8084 2129 15245 2904 25100 29386 8464 1628 25587 24672 26561 1135 22487 29377 13914 434 4174 3412 16837 1814 8376 23143 5224 4303 12196 23230 16515 12505 14432 3802 18561 26430 22120 3385 10065 25963 11991 21066 6516 15157 24320 27312 8442 23165 22001 21508 19755 28383 4546 3939 19406 15805 19355 295 13361 20136 23929 23888 4747 27605 8092 14750 15984 22104 16801 14001 5873 8306 29353 5110 3420 23991 6396 5895 6312 1444 13939 25798 15268 6622 5085 26399 15032 29052 13587 6463 17523 20446 8085 2130 14279 18138 740 2539 15246 3360 8964 24399 5521 4016 2905 1130 21077 27741 13865 2472 25101 10981 27485 24776 29387 21443 8465 10857 7696 11301 25982 7793 1629 19884 25588 100

In [113]:

"""Вариант с использованием библиотеки"""


sa = SuffixArray(sequence)

print("Суффиксный массив:")
print(*sa.suffix_array())
bwt = []
for i in range(len(sa.suffix_array())):

    idx = sa.suffix_array()[i]

    if idx == 0:
        bwt.append(chr(1))  
    else:
        bwt.append(sequence[idx - 1])

print("BWT:")
print(''.join(bwt))


Суффиксный массив:
29903 29902 29901 29900 29899 29898 29897 29896 29895 29894 29893 29892 29891 29890 29889 29888 29887 29886 29885 29884 29883 29882 29881 29880 29879 29878 29877 29876 29875 29874 29873 29872 29871 29870 1813 11990 23164 22000 13360 20135 23887 15983 16800 14000 25797 29051 13586 20445 8084 2129 15245 2904 25100 29386 8464 1628 25587 24672 26561 1135 22487 29377 13914 434 4174 3412 16837 1814 8376 23143 5224 4303 12196 23230 16515 12505 14432 3802 18561 26430 22120 3385 10065 25963 11991 21066 6516 15157 24320 27312 8442 23165 22001 21508 19755 28383 4546 3939 19406 15805 19355 295 13361 20136 23929 23888 4747 27605 8092 14750 15984 22104 16801 14001 5873 8306 29353 5110 3420 23991 6396 5895 6312 1444 13939 25798 15268 6622 5085 26399 15032 29052 13587 6463 17523 20446 8085 2130 14279 18138 740 2539 15246 3360 8964 24399 5521 4016 2905 1130 21077 27741 13865 2472 25101 10981 27485 24776 29387 21443 8465 10857 7696 11301 25982 7793 1629 19884 25588 1001 5497 24673 286

![correct](BWT_folder/BWT1.png)

## Task 2: Create FM index
- Construct Occurence array
- Construct Count dictionary
- Make a class BWTSearcher

In [156]:
from collections import defaultdict
class BWTSearcher:
    def __init__(self, seq):
        self.bwt = self.BWT(seq)
        self.C = self.count_dictionary(self.bwt)
        self.Occ = self.occurence(self.bwt)
    def count(self, pattern):
        l = 0
        r = len(self.bwt) - 1
        for ch in reversed(pattern):
            if ch not in self.C:
                return 0 
            
            l = self.C[ch] + self.Occ[ch][l]
            r = self.C[ch] + self.Occ[ch][r + 1] - 1
            print(l, r)
            if l > r:
                return 0
        return r - l + 1
    
    def BWT(self, sequence):
        sa = SuffixArray(sequence)
        bwt = []
        for i in range(len(sa.suffix_array())):

            idx = sa.suffix_array()[i]
            if idx == 0:
                bwt.append(chr(1))  
            else:
                bwt.append(sequence[idx - 1])

        return ''.join(bwt)

    def count_dictionary(self, bwt):
        un_ch = sorted(set(bwt))
        count = {}
        total = 0
        for c in un_ch:
            count[c] = total
            total += bwt.count(c)
        return count

    def occurence(self, bwt):
        occ = defaultdict(lambda: [0])
        # print(occ)
        
        al = sorted(set(bwt))
        for c in al:
            occ[c] = [0]
        for ch in bwt:
            for c in al:
                occ[c].append(occ[c][-1] + (1 if c == ch else 0))
        return occ

In [157]:
r = BWTSearcher("banana")

bwt = r.BWT("banana")
for i, v in r.occurence(bwt).items():
    print(i, v)
print(r.count_dictionary(bwt))


print(r.count("ana"))

 [0, 0, 0, 0, 0, 1, 1, 1]
a [0, 1, 1, 1, 1, 1, 2, 3]
b [0, 0, 0, 0, 1, 1, 1, 1]
n [0, 0, 1, 2, 2, 2, 2, 2]
{'\x01': 0, 'a': 1, 'b': 4, 'n': 5}
1 3
5 6
2 3
2


### Task 4:
- There are 100 reads that were randomly sampled from genome.fa
- Some of them are error free, some contain one mutation, and some contain 5 mutations
- Could you use your BWTSearcher class to classify them? Think about the solution and implement it. You can add any functions of class members
- How many reads of each class did you find?

In [3]:
from Bio import SeqIO
# Process each read through the BWTSearcher
with open("BWT_folder/sample_reads.fasta", "r") as file:
    for record in SeqIO.parse(file, "fasta"):
        read_sequence = str(record.seq)

        # Here is just a placeholder to demonstrate using the read with the BWTSearcher
        print("Processing read:", read_sequence)

Processing read: TATGGACAACTCACCTAATTTAGCATGGCCTCTTATTGTAACAGCTTTAAGGGCCAATCCTTCTGTAAAATTACAGAATAATGATCTTAGTCCTGTTGCA
Processing read: TTGTGTTCCCTTGAACATAATACCTCTTACAACAGCAGCCAAACTAATGGTTGTCATACCAGACTATAACACATATAAAAATACGTTTGATGGTACAACA
Processing read: AGTGTTACCACAGAGATTCTACCTGTTTCTATGACCAAGACATGAGTAGATTGTACAATGTACATTTGTGGTGATTCAACTGAATGCAGCAATCTCTTGT
Processing read: AAAGGGGTAAGGCTAGACTTTATAATGATTCAATGAGTTATGAGGATCAAGATGCACTTTTCGCATATACAACATGTAATGTCATCCCTACTATCACTCA
Processing read: TTTTGATAAATGCAAAGTGAATTCAACATTAGAACAGTATGTCTTTTGTACTGTAAATGCATTGCCTGAGACGACAGCAGATATAGTTGTCTTTGATGAA
Processing read: AGTTGGTAACATCTGTTACACACCATCAAAACTTATAGAGTACACTGACTTTGCAACATCAGCTTGTGTTTTGGCTGCTGAATGTACAATTTTTAAAGAT
Processing read: TTAAAACTCTTTGACCGTTATTTTAAATATTGGGATCAGACATACCACCCAAATTGTGTTAACTGTTTGGATCACAGATGCATTCTGCATTGTGCAAACT
Processing read: TGTTGAAGGTTTTAATTGTTACTTTCCTTTACAATCATAGGGTTTCCAACCCACTAATGGTGTTGGTTACCAACCATACAGAGTAGTAGTACTTTCTTTT
Processing read: TACAAATTAAATGTTGGTGATTATTTTGTGCTGACATCA

In [140]:
from collections import defaultdict
class BWTSearcher:
    def __init__(self, seq):
        self.ref_seq = seq
        self.bwt = self.BWT(seq)
        self.C = self.count_dictionary(self.bwt)
        self.Occ = self.occurence(self.bwt)
    def count(self, pattern):
        l = 0
        r = len(self.bwt) - 1
        for ch in reversed(pattern):
            if ch not in self.C:
                return 0  
            l = self.C[ch] + self.Occ[ch][l]
            r = self.C[ch] + self.Occ[ch][r + 1] - 1
            if l > r:
                return 0
        return r - l + 1
    
    def BWT(self, sequence):
        sa = SuffixArray(sequence)
        bwt = []
        for i in range(len(sa.suffix_array())):

            idx = sa.suffix_array()[i]
            if idx == 0:
                bwt.append(chr(1))  
            else:
                bwt.append(sequence[idx - 1])

        return ''.join(bwt)

    def count_dictionary(self, bwt):
        un_ch = sorted(set(bwt))
        count = {}
        total = 0
        for c in un_ch:
            count[c] = total
            total += bwt.count(c)
        return count

    def occurence(self, bwt):
        occ = defaultdict(lambda: [0])
        al = sorted(set(bwt))
        for c in al:
            occ[c] = [0]
        for ch in bwt:
            for c in al:
                occ[c].append(occ[c][-1] + (1 if c == ch else 0))
        return occ

    def one_mismatch_search(self, pattern):
        alphabet = ['A', 'C', 'G', 'T']
        L = len(pattern)
        pat_list = list(pattern)

        for i in range(L):
            orig = pat_list[i]
            for nt in alphabet:
                if nt == orig:
                    continue
                pat_list[i] = nt
                candidate = ''.join(pat_list)
                if self.count(candidate) > 0:
                    pat_list[i] = orig
                    return True
            pat_list[i] = orig
        return False

    def k_mismatch_search(self, pattern, k):
        L = len(pattern)
        num_blocks = k + 1
        base_size = L // num_blocks
        extra = L % num_blocks

        block_sizes = []
        for i in range(num_blocks):
            if i < extra:
                block_sizes.append(base_size + 1)
            else:
                block_sizes.append(base_size)

        blocks = []
        start = 0
        for size in block_sizes:
            blocks.append((start, start + size))
            start += size
        def get_all_positions_exact(sub_block):
            res = []
            pos = 0
            while True:
                idx = self.ref_seq.find(sub_block, pos)
                if idx == -1:
                    break
                res.append(idx)
                pos = idx + 1
            return res
        for j in range(num_blocks):
            (b_start, b_end) = blocks[j]
            sub_block = pattern[b_start:b_end]
            exact_positions = get_all_positions_exact(sub_block)
            if not exact_positions:
                continue
            for pos in exact_positions:
                ref_start = pos - b_start
                if ref_start < 0:
                    continue
                ref_end = ref_start + L
                if ref_end > len(self.ref_seq):
                    continue
                genome_segment = self.ref_seq[ref_start:ref_end]
                mismatches = 0
                for a, b in zip(genome_segment, pattern):
                    if a != b:
                        mismatches += 1
                        if mismatches > k:
                            break
                if mismatches == k:
                    return True
        return False


if __name__ == "__main__":
    ref_seq = ""
    for record in SeqIO.parse("BWT_folder/genome.fa", "fasta"):
        ref_seq += str(record.seq)

    bwts = BWTSearcher(ref_seq)
    count0 = 0
    count1 = 0
    count5 = 0

    for record in SeqIO.parse("BWT_folder/sample_reads.fasta", "fasta"):
        read_seq = str(record.seq)
        if bwts.count(read_seq) > 0:
            count0 += 1
            continue
        if bwts.one_mismatch_search(read_seq):
            count1 += 1
            continue
        if bwts.k_mismatch_search(read_seq, 5):
            count5 += 1
            continue
        print(f"Не удалось отнести рид {record.id} ни к одной из категорий 0/1/5 mismatches.")
    print("=== Итоговая классификация: ===")
    print(f"Риды с 0 мутаций: {count0}")
    print(f"Риды с 1 мутацией: {count1}")
    print(f"Риды с 5 мутациями: {count5}")

=== Итоговая классификация: ===
Риды с 0 мутаций: 45
Риды с 1 мутацией: 31
Риды с 5 мутациями: 24
